<a href="https://colab.research.google.com/github/Khaarl/VIZDOOM1/blob/main/DummyTESTFILE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Install required packages
from google.colab import drive
import os
import shutil
from tqdm import tqdm

def mount_drive():
    """Mount Google Drive and create test folder"""
    drive.mount('/content/drive')
    drive_folder = '/content/drive/MyDrive/DummyTestFiles'
    os.makedirs(drive_folder, exist_ok=True)
    return drive_folder

def validate_filename(name):
    """Check if filename is valid"""
    if not name:
        raise ValueError("Filename cannot be empty")
    # Check for invalid characters
    invalid_chars = set('/\\<>:"|?*')
    if any(char in invalid_chars for char in name):
        raise ValueError("Filename contains invalid characters")
    if name.startswith('.') or name.endswith('.'):
        raise ValueError("Filename cannot start or end with a dot")
    return name

def parse_size(size_str):
    """Convert MB size to bytes (accepts numbers or 'XMB' format)"""
    size_str = size_str.upper().replace('MB', '').strip()
    try:
        mb = float(size_str)
        if mb <= 0:
            raise ValueError("Size must be greater than 0")
        return int(mb * 1024**2)  # Convert MB to bytes
    except ValueError:
        raise ValueError("Invalid size format. Use numbers only (e.g., '200' or '200MB')")

def generate_dummy_file(full_path, target_bytes):
    """Generate dummy file with progress bar"""
    # Check available space
    dir_path = os.path.dirname(full_path) or '.'
    total, used, free = shutil.disk_usage(dir_path)

    if free < target_bytes:
        raise ValueError(f"Not enough space. Required: {target_bytes:,} bytes, Available: {free:,} bytes")

    # Generate file
    chunk_size = 1024 * 1024  # 1MB chunks
    with open(full_path, 'wb') as f:
        with tqdm(total=target_bytes, unit='B', unit_scale=True,
                 desc=f"Creating {os.path.basename(full_path)}") as pbar:
            bytes_written = 0
            while bytes_written < target_bytes:
                chunk = min(chunk_size, target_bytes - bytes_written)
                f.write(os.urandom(chunk))
                bytes_written += chunk
                pbar.update(chunk)

    # Verify size
    actual_size = os.path.getsize(full_path)
    if actual_size != target_bytes:
        os.remove(full_path)
        raise RuntimeError(f"Size mismatch. Requested: {target_bytes}, Actual: {actual_size}")

def main():
    # Mount Drive and prepare folder
    drive_folder = mount_drive()

    # Get valid filename
    while True:
        try:
            filename = input("Enter file name (e.g., 'test1.bin'): ").strip()
            validate_filename(filename)
            full_path = os.path.join(drive_folder, filename)
            break
        except ValueError as e:
            print(f"Invalid filename: {e}. Please try again.")

    # Get valid size
    while True:
        try:
            size_input = input("Enter file size in MB (e.g., '200' or '200MB'): ").strip()
            target_bytes = parse_size(size_input)
            break
        except ValueError as e:
            print(f"Invalid size: {e}. Please try again.")

    # Generate file
    try:
        generate_dummy_file(full_path, target_bytes)
        print(f"\nSuccess! File created at: {full_path}")
        print(f"Size: {os.path.getsize(full_path)/1024**2:.2f} MB")
    except Exception as e:
        print(f"\nError: {e}")
        if os.path.exists(full_path):
            os.remove(full_path)

if __name__ == "__main__":
    main()

Mounted at /content/drive
Enter file name (e.g., 'test1.bin'): test1.bin
Enter file size in MB (e.g., '200' or '200MB'): 200


Creating test1.bin: 100%|██████████| 210M/210M [00:01<00:00, 159MB/s]



Success! File created at: /content/drive/MyDrive/DummyTestFiles/test1.bin
Size: 200.00 MB
